# Training a mask/no mask classification model

Notebook to train a mask/no mask classification model.   
The input are images of faces that are already cropped. 

We plan to use this notebook with three kinds of images:  
    - Images of faces without mask  
    - Images of faces with an artificial mask  
    - Images of faces with a real mask

Note: for now we have done this with a sample from the MAFA dataset (http://www.escience.cn/people/geshiming/mafa.html).  
I have randomly marked half of the masked faces as being artificially masked, just to already get the code for the separate accuracies.  
As we'll plug in our own data later, no tuning is done yet and only minimal training.


We train a MobileNetV2 base followed by a Global Average Pooling layer, 2 fully connected layers and finally a sigmoid.

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import MobileNetV2
from keras.applications.mobilenet import MobileNet
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.applications.mobilenet import preprocess_input

import keras 

from sklearn import metrics 

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [2]:
target_size = (224,224)
batch_size = 32
lr = 0.01
n_epochs = 1

# csv files with paths to training and validation images
# these files contain a column with the class (masked or not_masked)
# they also contain a column that indicates if it is a real mask or artificially generated
train_csv = 'tmp_data/train.csv'
val_csv = 'tmp_data/val.csv'

In [3]:
base_model = MobileNet(weights='imagenet',include_top=False, input_shape=(224,224,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x) 
x = Dense(512,activation='relu')(x) 
x = Dense(256,activation='relu')(x) 
preds = Dense(1,activation='sigmoid')(x)

In [4]:
model = Model(inputs=base_model.input,outputs=preds)

In [5]:
# we only train the final fully connected layers, not the mobilenet base
for layer in model.layers[:-4]:
    layer.trainable = False

In [6]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_df = pd.read_csv(train_csv)
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    target_size=target_size,
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    classes=['masked', 'not_masked'],
                                                    shuffle=True,
                                                    x_col='image',
                                                    y_col='label')

Found 2000 validated image filenames belonging to 2 classes.


In [7]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_df = pd.read_csv(val_csv)
val_generator = val_datagen.flow_from_dataframe(val_df, 
                                                shuffle=False,
                                                target_size=target_size,
                                                batch_size=batch_size,
                                                class_mode='binary',
                                                classes=['masked', 'not_masked'],
                                                x_col='image',
                                                y_col='label')

Found 1000 validated image filenames belonging to 2 classes.


In [8]:
opt = keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
step_size_train = train_generator.n//train_generator.batch_size
step_size_val = val_generator.n//val_generator.batch_size

In [9]:
class SubsetAccuracy(keras.callbacks.Callback):
    """
    We want to monitor accuracy in the validation set separately for real and artificial face masks.
    This callback will print this to the output, and store the values for each epoch.
    """
    
    def __init__(self, val_gen=None, val_df=None):
        self.val_gen = val_gen
        self.val_df = val_df
        
        artificial_idx = val_df.index[val_df.artificial == "yes"].tolist()
        real_masked_idx = val_df.index[(val_df.artificial == "no") & (val_df.label == "masked")].tolist()
        non_masked_idx = val_df.index[val_df.label == "not_masked"].tolist()
        
        # the 'real' validation subset consists of all non-masked images, 
        # and all real masked images
        self.real_eval_idxs = non_masked_idx + real_masked_idx
        self.real_eval_ground_truth = [1] * len(non_masked_idx) + [0] * len(real_masked_idx)
        
        # the 'artificial' validation subset consists of all non-masked images,
        # and all artificial masked images
        self.artificial_eval_idxs = non_masked_idx + artificial_idx
        self.artificial_eval_ground_truth = [1] * len(non_masked_idx) + [0] * len(artificial_idx)
        
        # these will store accuracies for different sets after each epoch
        self.acc_all = []
        self.acc_real = []
        self.acc_artificial = []
        
    def on_epoch_end(self, batch, logs={}):
        pred = self.model.predict_generator(self.val_gen)
        pred = [x[0] > 0.5 for x in pred]

        accuracy_all = metrics.accuracy_score(self.val_gen.labels, pred)
        print(f"The accuracy for the full validation set is {accuracy_all:.3f}")
                
        real_eval_preds = [pred[idx] for idx in self.real_eval_idxs]
        accuracy_real = metrics.accuracy_score(self.real_eval_ground_truth, real_eval_preds)
        print(f"The accuracy for the validation set with only real masked faces is: {accuracy_real:.3f}")

        artifical_eval_preds = [pred[idx] for idx in self.artificial_eval_idxs]
        accuracy_artificial = metrics.accuracy_score(self.artificial_eval_ground_truth, artifical_eval_preds)
        print(f"The accuracy for the validation set with only artificial masked faces is: {accuracy_artificial:.3f}")
        
        self.acc_all.append(accuracy_all)
        self.acc_real.append(accuracy_real)
        self.acc_artificial.append(accuracy_artificial)
          
subset_acc = SubsetAccuracy(val_generator, val_df)

In [10]:
model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=n_epochs, validation_data=val_generator, validation_steps=step_size_val, callbacks=[subset_acc])

Epoch 1/1
62/62 [==============================] - 74s 1s/step - loss: 2.2648 - accuracy: 0.8155 - val_loss: 0.1554 - val_accuracy: 0.9335
The accuracy for the full validation set is 0.933
The accuracy for the validation set with only real masked faces is: 0.924
The accuracy for the validation set with only artificial masked faces is: 0.933
